In [ ]:
# ---------------------------------------------
# Phase‑1 · Data loading & feature preparation
# ---------------------------------------------
import pandas as pd
import numpy as np

# --- 1. Load the dataset ---
file_path = r'D:\IITR DS Final Year Thesis\Dataset Superstore\archive\generated_single_product_dataset_with_seasonal_variation.csv'
df = pd.read_csv(file_path)

# --- 2. Parse dates & sort chronologically ---
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date').reset_index(drop=True)

# --- 3. Verify / recreate seasonal indices ---
# If the CSV already contains these columns, this will simply overwrite with correct values
df['day_of_year'] = df['date'].dt.dayofyear
df['day_of_week'] = df['date'].dt.weekday          # Monday=0 … Sunday=6  (matches your earlier screenshots)

# --- 4. Annual seasonality terms (period = 365 days) ---
df['sin_annual'] = np.sin(2 * np.pi * df['day_of_year'] / 365.0)
df['cos_annual'] = np.cos(2 * np.pi * df['day_of_year'] / 365.0)

# --- 5. Weekly dummy variables (baseline = day 0) ---
weekly_dummies = pd.get_dummies(df['day_of_week'], prefix='dow', drop_first=True)
df = pd.concat([df, weekly_dummies], axis=1)

# --- 6. Optional quick check ---
print("DataFrame shape:", df.shape)
print(df.head())

# Keep a list of feature columns (intercept will be added later in the modelling block)
weekly_cols = [c for c in df.columns if c.startswith('dow_')]   # dow_1 … dow_6
static_feature_cols = weekly_cols + ['sin_annual', 'cos_annual']  # for reuse in later blocks

# ---------------------------------------------------------------
# Block 1 · Prior specification & model initialisation
# ---------------------------------------------------------------
#
# Goal:
#   • Define the complete feature set that drives demand
#   • Encode informative but weakly‑restrictive priors
#   • Instantiate the posterior mean / covariance that will be
#     updated transaction‑by‑transaction in Block 3
#   • Record a few handy constants for later use
#
# Assumes that Block 0 has already created:
#   - df                       ← the cleaned DataFrame
#   - static_feature_cols      ← weekly dummies + ['sin_annual', 'cos_annual']
#     (intercept and price will be appended below)

import numpy as np

# ────────────────────────────────────────────────────────────────
# 1.  Assemble feature list  (order matters – keep it consistent)
# ────────────────────────────────────────────────────────────────
# Rename price column once for convenience
if 'price' not in df.columns:
    df['price'] = df['price_paid']          #   ← demand model uses this name

feature_cols   = ['intercept'] + static_feature_cols + ['price']
idx_price      = len(feature_cols) - 1      # index of the price coefficient
n_features     = len(feature_cols)

# ────────────────────────────────────────────────────────────────
# 2.  Set prior means  (μ₀)  — weakly informative
# ────────────────────────────────────────────────────────────────
prior_mean = np.zeros(n_features)

# Intercept: start around the median quantity sold
prior_mean[0] = df['quantity'].median()     # e.g. ~25–30 units

# Weekly / annual terms default to 0  (no seasonal bias initially)

# Price coefficient: strongly expected to be negative
prior_mean[idx_price] = -1.0                # units: Δquantity / Δprice

# ────────────────────────────────────────────────────────────────
# 3.  Set prior covariance  (Σ₀ = diag(σ²))
#     larger σ² ⇒ weaker prior confidence
# ────────────────────────────────────────────────────────────────
prior_sd = np.ones(n_features) * 10.0       # default ±10 units on coeffs
prior_sd[idx_price] = 1.0                   # tighter on price slope

prior_cov = np.diag(prior_sd ** 2)          # (n_features × n_features)

# ────────────────────────────────────────────────────────────────
# 4.  Observation noise variance  (σ²_ε)
#     Use sample std of quantity as a rough starting point
# ────────────────────────────────────────────────────────────────
sigma = df['quantity'].std()                # ≈ 12–15 for your synthetic data
sigma2 = sigma ** 2

# ────────────────────────────────────────────────────────────────
# 5.  Initialise posterior as the prior
#     (will be updated online in Block 3)
# ────────────────────────────────────────────────────────────────
post_mean = prior_mean.copy()
post_cov  = prior_cov.copy()

# ────────────────────────────────────────────────────────────────
# 6.  Convenience constants
# ────────────────────────────────────────────────────────────────
price_bounds   = (df['price'].min(), df['price'].max())   # for clamping TS prices
cost_per_unit  = df['cost'].mean()                        # will be used for profit calc

# ────────────────────────────────────────────────────────────────
# 7.  Quick sanity printout
# ────────────────────────────────────────────────────────────────
print("➤ Feature columns:", feature_cols)
print("➤ Prior mean:", prior_mean.round(2))
print("➤ Prior SD  :", prior_sd.round(2))
print(f"➤ Obs‑noise σ² ≈ {sigma2:.2f}")


# ---------------------------------------------------------------
# Block 2 · Utility helpers
# ---------------------------------------------------------------
#
# Pure‑Python functions that keep the main loop tidy.
# They assume these variables already exist in the workspace:
#   • weekly_cols       – list like ['dow_1', …, 'dow_6']
#   • price_bounds      – tuple (p_min, p_max)  from Block 1
#   • cost_per_unit     – scalar, average cost  (can be overridden)
#   • sigma2            – observation‑noise variance
#
# You can import this module in a larger project, or simply
# execute the cell once before running Block 3.
# ---------------------------------------------------------------

import numpy as np

# ────────────────────────────────────────────────────────────────
# 1.  Feature‑vector builders
# ────────────────────────────────────────────────────────────────
def build_static_vector(row, weekly_cols=weekly_cols):
    """
    Returns the *static* part of the feature vector for a single
    transaction (no price term):
        [1, dow_1 … dow_6, sin_annual, cos_annual]
    """
    vec = [1.0]                                         # intercept
    vec.extend([row.get(col, 0.0) for col in weekly_cols])
    vec.append(row['sin_annual'])
    vec.append(row['cos_annual'])
    return np.array(vec, dtype=float)                   # shape (len(weekly_cols)+3,)

def make_feature_vector(row, weekly_cols=weekly_cols):
    """
    Complete feature vector including the *actual* price in the row.
    """
    static_vec = build_static_vector(row, weekly_cols)
    return np.concatenate([static_vec, [row['price']]]) # final len = n_features


# ────────────────────────────────────────────────────────────────
# 2.  Price‑optimisation under a sampled θ (Thompson step)
# ────────────────────────────────────────────────────────────────
def optimal_price_thompson(theta, static_vec,
                           price_bounds=price_bounds,
                           cost=cost_per_unit):
    """
    Given:
        θ         – sampled coefficient vector  (len = n_features)
        static_vec– context WITHOUT price term  (same length as θ‑1)
    returns the profit‑maximising price P* within price_bounds.

    Demand model:  Q = A + B·P
       A = θ_static · static_vec
       B = θ_price
    Profit:        π(P) = (P - cost) * Q

    Closed‑form maximiser (if B < 0):
        P* = (cost·B - A) / (2·B)

    Fallbacks:
        • B == 0 → pick lower bound (explore low price)
        • B  > 0 → pick upper bound (monotonic violation → safe edge)
    """
    A = float(np.dot(theta[:-1], static_vec))
    B = float(theta[-1])

    if B < 0:                                           # expected case
        p_star = (cost * B - A) / (2.0 * B)
        # Numeric guard
        if not np.isfinite(p_star):
            p_star = 0.5 * (price_bounds[0] + price_bounds[1])
    elif B == 0:
        p_star = price_bounds[0]
    else:                                               # B > 0 (unlikely after learning)
        p_star = price_bounds[1]

    # Clamp to observed range
    p_star = max(price_bounds[0], min(price_bounds[1], p_star))
    return p_star


# ────────────────────────────────────────────────────────────────
# 3.  One‑step Bayesian update (Kalman style)
# ────────────────────────────────────────────────────────────────
def kalman_update(mean, cov, x_vec, y_obs, sigma2=sigma2):
    """
    Perform the conjugate Bayesian update for linear‑Gaussian
    regression on a single (x, y) observation.

    Args
    ----
      mean  : current posterior mean      (n,)
      cov   : current posterior covariance(n,n)
      x_vec : feature vector              (n,)
      y_obs : scalar target
      sigma2: observation noise variance

    Returns
    -------
      new_mean, new_cov   – updated posterior parameters
    """
    x = x_vec.reshape(-1, 1)                            # (n,1)
    S = sigma2 + float(x.T @ cov @ x)                   # predictive var
    K = (cov @ x) / S                                   # Kalman gain  (n,1)
    residual = y_obs - float(mean @ x)                  # innovation
    new_mean = mean + (K.flatten() * residual)          # (n,)
    new_cov  = cov - K @ (x.T @ cov)                    # rank‑1 update
    # Enforce symmetry
    new_cov  = 0.5 * (new_cov + new_cov.T)
    return new_mean, new_cov


# ────────────────────────────────────────────────────────────────
# 4.  Tiny helper for clamping generic scalars
# ────────────────────────────────────────────────────────────────
def clamp(val, lo, hi):
    return max(lo, min(hi, val))

# ------------------------------------------------------------------
# Block 3 · Online BLR + Thompson‑Sampling loop
# ------------------------------------------------------------------
# Prerequisites  (make sure Blocks 0‑2 ran first):
#   • df                 – time‑sorted DataFrame
#   • feature_cols       – full feature list (Block 1)
#   • post_mean, post_cov, sigma2, idx_price
#   • build_static_vector(), optimal_price_thompson(), kalman_update()
#
# What this block does
#   1. Pre‑computes static context vectors (fast lookup)
#   2. Iterates through all 10 000 transactions in sequence
#   3. For each tx:
#        • one‑step‑ahead demand prediction
#        • Thompson draw  → recommended price
#        • log actuals & profit
#        • Bayesian (Kalman) update of posterior
#   4. Stores *every* series you’ll need for diagnostics /
#      visualisation in the next blocks, and pickles them to disk.
# ------------------------------------------------------------------

import numpy as np
from tqdm import tqdm
import pickle
import os

# ────────────────────────────────────────────────────────────────
# 1.  Pre‑compute static context matrix  (no price term)
# ────────────────────────────────────────────────────────────────
static_matrix = np.stack(df.apply(build_static_vector, axis=1).values)  # shape (N, len(static)+1)
prices        = df['price'].values
quantities    = df['quantity'].values
N             = len(df)

# ────────────────────────────────────────────────────────────────
# 2.  Prepare logging containers
# ────────────────────────────────────────────────────────────────
rec_prices          = np.empty(N)                # Thompson‑recommended
act_prices          = prices                     # alias; kept for symmetry
act_qty             = quantities
pred_qty            = np.empty(N)                # one‑step‑ahead predictions
post_means_price    = np.empty(N)                # trace of β_price mean
profits             = np.empty(N)
cum_profit          = np.empty(N)
residuals           = np.empty(N)
theta_trace         = np.empty((N, len(feature_cols)))  # posterior mean each step

# ────────────────────────────────────────────────────────────────
# 3.  Online loop
# ────────────────────────────────────────────────────────────────
pm = post_mean.copy()     # start from prior (Block 1)
pc = post_cov.copy()

for i in tqdm(range(N), desc="Online BLR updating", ncols=80):

    # ------ context & actual outcome ----------------------------------
    static_vec = static_matrix[i]                # (len(static)+1,)
    price_i    = prices[i]
    qty_i      = quantities[i]

    # ------ one‑step‑ahead prediction (using current posterior mean) ---
    x_i = np.concatenate([static_vec, [price_i]])
    y_hat = float(pm @ x_i)
    pred_qty[i] = y_hat
    residuals[i] = qty_i - y_hat

    # ------ Thompson‑sampling price recommendation --------------------
    theta_sample = np.random.multivariate_normal(pm, pc)
    p_star = optimal_price_thompson(theta_sample, static_vec)
    rec_prices[i] = p_star

    # ------ Profit realised with the *actual* historical price --------
    profit_i = (price_i - cost_per_unit) * qty_i
    profits[i] = profit_i
    cum_profit[i] = profit_i if i == 0 else cum_profit[i-1] + profit_i

    # ------ Log posterior mean trace ----------------------------------
    post_means_price[i] = pm[idx_price]
    theta_trace[i] = pm

    # ------ Bayesian (Kalman) update with observed (x_i, qty_i) -------
    pm, pc = kalman_update(pm, pc, x_i, qty_i)   # returns updated mean & cov

# ────────────────────────────────────────────────────────────────
# 4.  Final posterior snapshot
# ────────────────────────────────────────────────────────────────
post_mean_final = pm.copy()
post_cov_final  = pc.copy()

print(f"\nTraining complete.  Final β_price mean = {post_mean_final[idx_price]:.4f}")

# ────────────────────────────────────────────────────────────────
# 5.  Persist logs so you can reload later without rerun
# ────────────────────────────────────────────────────────────────
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)
logs_path = os.path.join(results_dir, "phase1_logs.pkl")

logs = {
    "feature_cols"      : feature_cols,
    "recommended_price" : rec_prices,
    "actual_price"      : act_prices,
    "actual_quantity"   : act_qty,
    "predicted_quantity": pred_qty,
    "price_coef_mean"   : post_means_price,
    "profit"            : profits,
    "cum_profit"        : cum_profit,
    "residuals"         : residuals,
    "theta_trace"       : theta_trace,
    "post_mean_final"   : post_mean_final,
    "post_cov_final"    : post_cov_final,
    "sigma2"            : sigma2,
    "price_bounds"      : price_bounds,
    "cost_per_unit"     : cost_per_unit,
}

with open(logs_path, "wb") as f:
    pickle.dump(logs, f)

print(f"➤ Logs saved to {logs_path}")


# ------------------------------------------------------------------
# Block 4 · Persist / reload intermediate results
# ------------------------------------------------------------------
# Purpose
#   • Store the logged arrays from Block 3 in *human‑friendly*
#     formats (CSV, NPZ, JSON) so you can reload or inspect them
#     without rerunning the full online loop.
#   • Provide a tiny helper to load everything back in one call.
#
# Usage
#   • Run this cell *after* Block 3 (the ‘logs’ dict must exist),
#     OR let it auto‑load the default pickle if you started a
#     fresh session and haven’t recreated logs in memory yet.
# ------------------------------------------------------------------

import os, pickle, json, numpy as np, pandas as pd
from datetime import datetime

# ────────────────────────────────────────────────────────────────
# 1.  Locate / load the logs dict
# ────────────────────────────────────────────────────────────────
try:
    logs  # is it already in RAM?
except NameError:
    default_pkl = os.path.join("results", "phase1_logs.pkl")
    if not os.path.isfile(default_pkl):
        raise FileNotFoundError(
            "No in‑memory ‘logs’ and default pickle not found. "
            "Run Block 3 first or point to a specific pickle."
        )
    with open(default_pkl, "rb") as f:
        logs = pickle.load(f)
    print(f"✓ Loaded logs from {default_pkl}")

# ────────────────────────────────────────────────────────────────
# 2.  Create a unique results sub‑folder
# ────────────────────────────────────────────────────────────────
timestamp   = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir    = os.path.join("results", f"phase1_{timestamp}")
os.makedirs(base_dir, exist_ok=True)

# ────────────────────────────────────────────────────────────────
# 3.  Transaction‑level CSV  (easiest to eyeball in Excel/R)
# ────────────────────────────────────────────────────────────────
df_logs = pd.DataFrame({
    "recommended_price" : logs["recommended_price"],
    "actual_price"      : logs["actual_price"],
    "actual_quantity"   : logs["actual_quantity"],
    "predicted_quantity": logs["predicted_quantity"],
    "profit"            : logs["profit"],
    "cum_profit"        : logs["cum_profit"],
    "beta_price_mean"   : logs["price_coef_mean"],
})
csv_path = os.path.join(base_dir, "transaction_logs.csv")
df_logs.to_csv(csv_path, index_label="transaction_index")
print(f"✓ CSV saved → {csv_path}")

# ────────────────────────────────────────────────────────────────
# 4.  θ‑trace (posterior mean of *all* coeffs at every step)
#     → compressed NPZ to keep file size modest
# ────────────────────────────────────────────────────────────────
theta_path = os.path.join(base_dir, "theta_trace.npz")
np.savez_compressed(theta_path, theta=logs["theta_trace"])
print(f"✓ θ‑trace saved → {theta_path}")

# ────────────────────────────────────────────────────────────────
# 5.  Final posterior arrays for warm‑starts or later analysis
# ────────────────────────────────────────────────────────────────
np.save(os.path.join(base_dir, "beta_mean_final.npy"), logs["post_mean_final"])
np.save(os.path.join(base_dir, "beta_cov_final.npy"),  logs["post_cov_final"])
print("✓ Final posterior mean / cov saved (.npy)")

# ────────────────────────────────────────────────────────────────
# 6.  Lightweight JSON meta‑info
# ────────────────────────────────────────────────────────────────
meta = {
    "timestamp"       : timestamp,
    "feature_cols"    : logs["feature_cols"],
    "sigma2"          : float(logs["sigma2"]),
    "price_bounds"    : [float(x) for x in logs["price_bounds"]],
    "cost_per_unit"   : float(logs["cost_per_unit"]),
    "n_transactions"  : int(len(logs["actual_price"])),
}
meta_path = os.path.join(base_dir, "meta.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)
print(f"✓ Meta‑info saved → {meta_path}")

# ────────────────────────────────────────────────────────────────
# 7.  Handy loader for future notebooks / scripts
# ────────────────────────────────────────────────────────────────
def load_phase1_results(folder):
    """
    Reloads everything produced by this block.
    
    Returns
    -------
      df_logs       – pandas DataFrame (transaction‑level log)
      theta_trace   – np.ndarray (N, n_features)
      beta_mean_fin – np.ndarray (n_features,)
      beta_cov_fin  – np.ndarray (n_features, n_features)
      meta          – dict
    """
    df_logs = pd.read_csv(os.path.join(folder, "transaction_logs.csv"),
                          index_col="transaction_index")
    theta   = np.load(os.path.join(folder, "theta_trace.npz"))["theta"]
    beta_m  = np.load(os.path.join(folder, "beta_mean_final.npy"))
    beta_c  = np.load(os.path.join(folder, "beta_cov_final.npy"))
    with open(os.path.join(folder, "meta.json"), "r") as f:
        meta = json.load(f)
    return df_logs, theta, beta_m, beta_c, meta

print("\nHelper ready → load_phase1_results(<folder>)")
print("All artefacts safely stored.")


# ------------------------------------------------------------------
# Block 5 · Visualisation suite
# ------------------------------------------------------------------
# Six ready‑to‑run plots:
#   5‑a  Demand curve vs. observed data   (★ most important)
#   5‑b  Predicted vs actual demand over time
#   5‑c  Cumulative profit over time
#   5‑d  Price trajectory  (actual vs TS‑recommended)
#   5‑e  Posterior mean of β_price over time
#   5‑f  Histogram of one‑step‑ahead residuals
#
# Assumptions
#   • Either the logs dict is still in RAM from Block 3/4 **or**
#     you point results_folder at one of the saved folders
#     created by Block 4.  All plots save a PNG alongside display.
# ------------------------------------------------------------------

import os, pickle, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────────────────────────────────────────
# 0.  Load results (if not already in memory)
# ────────────────────────────────────────────────────────────────
results_folder = None   # ⇠ set to e.g. "results/phase1_20250505_211530"
                        #    or leave None to use in‑RAM logs

if results_folder is None:
    try:
        logs
    except NameError:
        raise RuntimeError("logs dict not found in memory and "
                           "results_folder is None.  "
                           "Either run Blocks 3‑4 or set results_folder.")
    meta = None  # not needed when logs already in memory
    df_logs = pd.DataFrame({
        "recommended_price" : logs["recommended_price"],
        "actual_price"      : logs["actual_price"],
        "actual_quantity"   : logs["actual_quantity"],
        "predicted_quantity": logs["predicted_quantity"],
        "profit"            : logs["profit"],
        "cum_profit"        : logs["cum_profit"],
        "beta_price_mean"   : logs["price_coef_mean"],
    })
    theta_trace        = logs["theta_trace"]
    post_mean_final    = logs["post_mean_final"]
    price_bounds       = logs["price_bounds"]
else:
    # Use helper from Block 4
    df_logs, theta_trace, post_mean_final, _, meta = load_phase1_results(results_folder)
    price_bounds = meta["price_bounds"]

# Ensure an output dir for plots
plots_dir = os.path.join(results_folder if results_folder else "results", "plots")
os.makedirs(plots_dir, exist_ok=True)

# Short aliases
N = len(df_logs)
price_min, price_max = price_bounds

# ---------------------------------------------------------------
# 5‑a  Estimated demand curve (baseline context) vs observed data
# ---------------------------------------------------------------
# Baseline context = day_of_week 0 (all dummies 0), sin=0, cos=1
n_static = len(post_mean_final) - 1
baseline_static = np.zeros(n_static)
baseline_static[0] = 1.0                     # intercept is first static entry

price_grid = np.linspace(price_min, price_max, 250)
q_pred_curve = post_mean_final[:-1] @ baseline_static + post_mean_final[-1] * price_grid

plt.figure(figsize=(7,4.5))
# scatter – sample for clarity if dataset is huge
sample_idx = np.random.choice(N, size=min(3000, N), replace=False)
plt.scatter(df_logs["actual_price"].iloc[sample_idx],
            df_logs["actual_quantity"].iloc[sample_idx],
            s=14, alpha=0.25, label="Observed transactions")

plt.plot(price_grid, q_pred_curve, color="darkorange", linewidth=2.5,
         label="Final estimated demand curve\n(baseline context)")

plt.xlabel("Price")
plt.ylabel("Quantity demanded")
plt.title("Estimated Demand Curve vs Observed Data")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "demand_curve.png"), dpi=150)
plt.show()

# ---------------------------------------------------------------
# 5‑b  Predicted vs actual demand over time
# ---------------------------------------------------------------
plt.figure(figsize=(10,4))
plt.plot(df_logs["actual_quantity"].values, label="Actual qty", linewidth=1.1)
plt.plot(df_logs["predicted_quantity"].values, label="Predicted qty (1‑step ahead)",
         linestyle="--", linewidth=1.1)
plt.xlabel("Transaction index (chronological)")
plt.ylabel("Quantity")
plt.title("Predicted vs Actual Demand Over Time")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "pred_vs_actual.png"), dpi=150)
plt.show()

# ---------------------------------------------------------------
# 5‑c  Cumulative profit over time
# ---------------------------------------------------------------
plt.figure(figsize=(7,4))
plt.plot(df_logs["cum_profit"].values, color="seagreen")
plt.xlabel("Transaction index")
plt.ylabel("Cumulative profit")
plt.title("Cumulative Profit Curve")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "cumulative_profit.png"), dpi=150)
plt.show()

# ---------------------------------------------------------------
# 5‑d  Price trajectory: actual vs TS‑recommended
# ---------------------------------------------------------------
plt.figure(figsize=(10,4))
plt.plot(df_logs["actual_price"].values, label="Actual price", alpha=0.7, linewidth=1)
plt.plot(df_logs["recommended_price"].values, label="TS‑recommended price", linewidth=1)
plt.xlabel("Transaction index")
plt.ylabel("Price")
plt.title("Price Trajectory – Actual vs Thompson‑Sampling Recommendation")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "price_trajectory.png"), dpi=150)
plt.show()

# ---------------------------------------------------------------
# 5‑e  Posterior mean of β_price over time
# ---------------------------------------------------------------
plt.figure(figsize=(7,4))
plt.plot(df_logs["beta_price_mean"].values, color="firebrick")
plt.axhline(0, color="k", linewidth=0.8)
plt.xlabel("Transaction index")
plt.ylabel("Posterior mean of β_price")
plt.title("Convergence of Price‑Slope Coefficient")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "beta_price_trace.png"), dpi=150)
plt.show()

# ---------------------------------------------------------------
# 5‑f  Histogram of one‑step residuals
# ---------------------------------------------------------------
plt.figure(figsize=(6,4))
plt.hist(df_logs["actual_quantity"] - df_logs["predicted_quantity"],
         bins=60, alpha=0.8, edgecolor="k")
plt.xlabel("Residual (actual − predicted)")
plt.ylabel("Frequency")
plt.title("Distribution of 1‑Step‑Ahead Prediction Residuals")
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "residual_hist.png"), dpi=150)
plt.show()

print(f"✓ All plots saved to {plots_dir}")


# ══════════════════════════════════════════════════════════════
# 5‑g  Thompson‑Sampling realisation & optimal‑price illustration
# ══════════════════════════════════════════════════════════════
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

def plot_ts_realization(idx, K=40, price_points=200, save=True):
    """
    Recreates a Figure‑4 style illustration for transaction idx.
    
    Parameters
    ----------
    idx           : int   – index of the transaction (chronological order)
    K             : int   – how many posterior samples to visualise as 'uncertainty'
    price_points  : int   – resolution of the price grid
    save          : bool  – if True, PNG saved to plots_dir
    
    Produces
    --------
      A 3‑panel Matplotlib figure:
         (1) Posterior mean demand + K grey samples   → 'Uncertainty'
         (2) One Thompson sample highlighted in green
         (3) Revenue curve for that sample with p*τ   → dotted red vs green
    """
    assert 0 <= idx < N, "idx out of range"
    
    # ---------------- context & posterior ----------------------
    row          = df.iloc[idx]
    static_vec   = build_static_vector(row)
    cost         = cost_per_unit                           # mean cost from earlier
    
    # Use *final* posterior for uncertainty envelope (simpler)
    beta_mean    = post_mean_final
    beta_cov     = post_cov_final
    
    # Price grid for curves
    p_grid = np.linspace(price_min, price_max, price_points)
    
    # -- Posterior mean demand curve (baseline prediction) -----
    A_mean = beta_mean[:-1] @ static_vec
    B_mean = beta_mean[-1]
    q_mean_curve = A_mean + B_mean * p_grid
    
    # -- K posterior samples to illustrate uncertainty ---------
    theta_samples = np.random.multivariate_normal(beta_mean, beta_cov, size=K)
    q_samples = np.outer(theta_samples[:, :-1] @ static_vec, np.ones_like(p_grid)) \
                + (theta_samples[:, -1][:, None] * p_grid)
    
    # -- Thompson draw for idx (these days we simply draw anew)--
    theta_ts = np.random.multivariate_normal(beta_mean, beta_cov)
    A_ts, B_ts = theta_ts[:-1] @ static_vec, theta_ts[-1]
    q_ts_curve = A_ts + B_ts * p_grid
    if B_ts < 0:
        p_opt_ts = (cost * B_ts - A_ts) / (2 * B_ts)
        p_opt_ts = max(price_min, min(price_max, p_opt_ts))
    else:
        p_opt_ts = price_max if B_ts > 0 else price_min
    
    # Historical & recommended price from our logs
    p_actual = df_logs["actual_price"].iloc[idx]
    p_rec    = df_logs["recommended_price"].iloc[idx]
    
    # ---------------- revenue curve for the TS sample ----------
    revenue_ts = (p_grid - cost) * q_ts_curve
    
    # =====================  PLOT  ==============================
    fig = plt.figure(figsize=(11,3.4))
    gs  = GridSpec(1, 3, wspace=0.28)
    
    # Panel 1 – Uncertainty band
    ax1 = fig.add_subplot(gs[0,0])
    for k in range(K):
        ax1.plot(p_grid, q_samples[k], color="lightgrey", linewidth=0.8)
    ax1.plot(p_grid, q_mean_curve, color="black", linewidth=2,
             label="Posterior mean")
    ax1.set_title("Posterior demand & uncertainty")
    ax1.set_xlabel("Price  p");  ax1.set_ylabel("Quantity  v")
    ax1.legend(frameon=False)
    
    # Panel 2 – Thompson sample realisation
    ax2 = fig.add_subplot(gs[0,1])
    for k in range(K):
        ax2.plot(p_grid, q_samples[k], color="lightgrey", linewidth=0.8)
    ax2.plot(p_grid, q_ts_curve, color="forestgreen", linewidth=2,
             label="TS sample")
    ax2.set_title("Thompson‑sampled demand curve")
    ax2.set_xlabel("Price  p");  ax2.set_yticks([])
    ax2.legend(frameon=False)
    
    # Panel 3 – Objective (revenue) & p*τ marker
    ax3 = fig.add_subplot(gs[0,2])
    ax3.plot(p_grid, revenue_ts, color="crimson", linestyle="--",
             label="Revenue curve  (objective)")
    ax3.axvline(p_opt_ts, color="black", linewidth=2,
                label=f"p*τ  (TS optimum ≈ {p_opt_ts:,.1f})")
    ax3.scatter([p_actual], [0], color="steelblue", zorder=4,
                label=f"Historical price = {p_actual:,.1f}")
    ax3.scatter([p_rec], [0], color="purple", zorder=4,
                label=f"Our rec. price = {p_rec:,.1f}")
    ax3.set_ylabel("Revenue  v·(p−c)");  ax3.set_xlabel("Price  p")
    ax3.set_title("Objective & optimal price")
    ax3.legend(frameon=False, loc="upper right")
    
    fig.suptitle(f"TS Figure – transaction τ = {idx}", y=1.03, fontsize=14)
    fig.tight_layout()
    
    if save:
        fname = os.path.join(plots_dir, f"ts_figure_tau_{idx}.png")
        fig.savefig(fname, dpi=150, bbox_inches="tight")
        print(f"✓ Figure saved → {fname}")
    plt.show()

# --------------------  Example usage  ---------------------------
# plot_ts_realization(5000)   # choose any τ you’re curious about

In [ ]:
# ════════════════════════════════════════════════════════════════
#  SNAPSHOT CURRENT MODEL‑1 RESULTS ⇒  results_model_1
#  ▸ Run this RIGHT AFTER Model‑1’s training/persistence blocks.
#  ▸ No re‑loading needed – we just point to the in‑memory objects.
#  ▸ Automatic fallback: if key objects are missing (e.g. after
#    a kernel restart) we reload the most recent phase1_* folder.
# ════════════════════════════════════════════════════════════════
import os
from glob import glob
import pandas as pd
import numpy as np
import pprint

# ────────────────────────────────────────────────────────────────
# 1.  Detect whether key objects are already in RAM
#     (they are created by Model‑1’s Block 3 & Block 4)
# ────────────────────────────────────────────────────────────────
have_in_memory = all(name in globals() for name in [
    "logs",            # dict with raw arrays
    "df_logs",         # pretty transaction DataFrame
    "theta_trace",     # posterior mean path
    "post_mean_final", # final μ
    "post_cov_final",  # final Σ
])

if have_in_memory:
    print("✓ Found Model‑1 artefacts in memory – no disk reload needed.")
    model1_folder = globals().get("base_dir", "") or "‑in‑memory‑run‑"
    df_logs1      = df_logs
    theta1        = theta_trace
    mu1           = post_mean_final
    Sigma1        = post_cov_final
    meta1         = {
        "feature_cols" : logs["feature_cols"],
        "price_bounds" : logs["price_bounds"],
        "sigma2"       : logs["sigma2"],
        "cost_per_unit": logs["cost_per_unit"],
        "folder"       : model1_folder
    }

else:
    # ────────────────────────────────────────────────────────────
    # 2.  Fallback – reload latest phase1_* folder from ./results
    # ────────────────────────────────────────────────────────────
    try:
        load_phase1_results
    except NameError as err:
        raise NameError("Model‑1 objects not in RAM and helper "
                        "`load_phase1_results()` is missing. "
                        "Run Model‑1’s persistence block first.") from err

    phase_dirs = sorted(
        glob(os.path.join("results", "phase1_*")),
        key=os.path.getmtime
    )
    if not phase_dirs:
        raise FileNotFoundError("No phase1_* folders found in ./results/. "
                                "Run Model‑1 first.")
    model1_folder = phase_dirs[-1]            # newest folder
    print(f"ℹ Objects not in RAM  – reloading from  {model1_folder}")
    df_logs1, theta1, mu1, Sigma1, meta1 = load_phase1_results(model1_folder)

# ────────────────────────────────────────────────────────────────
# 3.  Bundle everything into one convenient dict
# ────────────────────────────────────────────────────────────────
results_model_1 = {
    "folder"     : model1_folder,
    "df_logs"    : df_logs1,      # per‑transaction DataFrame
    "theta_trace": theta1,        # (N, d) array
    "mu_final"   : mu1,           # final posterior mean
    "Sigma_final": Sigma1,        # final posterior covariance
    "meta"       : meta1          # miscellaneous meta‑info
}

print("\nresults_model_1 is now ready with keys:")
pprint.pp(results_model_1.keys())
print(f"\n↳ Total profit recorded by Model‑1 = "
      f"{results_model_1['df_logs']['profit'].sum():,.2f}")
